In [1]:
from model import ss_fusion_cls
import torch
from torch  import nn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,cohen_kappa_score
from model import split_data,utils
from sklearn import metrics, preprocessing
from mmengine.optim import build_optim_wrapper
from mmcv_custom import custom_layer_decay_optimizer_constructor,layer_decay_optimizer_constructor_vit
import scipy.io as sio
from thop import profile
from multiprocessing import shared_memory

/home/wsq/anaconda3/envs/hypersigma/lib/python3.8/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/wsq/anaconda3/envs/hypersigma/lib/python3.8/site-packages/mmengine/optim/optimizer/zero_optimizer.py:11: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import \


In [2]:
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
#device = torch.device("cuda")
print(device)

cuda:2


In [3]:
class DataReader():
    def __init__(self):
        self.data_cube = None
        self.g_truth = None

    @property
    def cube(self):
        """
        origin data
        """
        return self.data_cube

    @property
    def truth(self):
        return self.g_truth

    @property
    def normal_cube(self):
        """
        normalization data: range(0, 1)
        """
        return (self.data_cube - np.min(self.data_cube)) / (np.max(self.data_cube) - np.min(self.data_cube))
class IndianRaw(DataReader):
    def __init__(self):
        super(IndianRaw, self).__init__()
        raw_data_package = sio.loadmat(r"data/Indian_pines_corrected.mat")
        self.data_cube = raw_data_package["data"].astype(np.float32)
        truth = sio.loadmat(r"data/Indian_pines_gt.mat")
        self.g_truth = truth["groundT"].astype(np.float32)

In [4]:
def load_data():
    data = IndianRaw().cube
    data_gt = IndianRaw().truth
    return data, data_gt


In [5]:
img_size =33
patch_size = 2
pca_components = 30
split_type = ['number', 'ratio'][0]
train_num =10
val_num =5
train_ratio = 0.98 
val_ratio = 0.01 

max_epoch = 100
batch_size = 16
learning_rate = 0.00001 
dataset_name = 'HH'

path_weight = r"/home5/wsq/HyperSIGMA/ImageClassification/weights/"
path_result = r"/home5/wsq/HyperSIGMA/ImageClassification/result/"
data, data_gt = load_data()
height, width, bands = data.shape
gt_reshape = np.reshape(data_gt, [-1])
class_num = np.max(data_gt)
class_num = class_num.astype(int)

In [6]:
data, data_gt = load_data()
train_index, val_index, test_index = split_data.split_data(gt_reshape, 
            class_num, train_ratio, train_ratio, train_num, val_num, split_type)


In [7]:
data, pca = split_data.apply_PCA(data, num_components=pca_components)
data_all, data_label_all = split_data.create_patches(data, data_gt, window_size=img_size, remove_zero_labels = False)

In [8]:
train_index=train_index.astype(int)
val_index=val_index.astype(int)
test_index=test_index.astype(int)
height, width, bands = data.shape
gt_reshape = np.reshape(data_gt, [-1])
class_num = np.max(gt_reshape)
class_num = class_num.astype(int)

In [9]:
train_index0, val_index0, re_index = split_data.split_data(gt_reshape, 
            class_num, 0, 0, 0, 10,'ratio' )
re_index = np.sort(re_index)

train_index = train_index.reshape(-1,1)
val_index = val_index.reshape(-1,1)
test_index = test_index.reshape(-1,1)

train_label = data_label_all[train_index].reshape(-1,)
val_label = data_label_all[val_index].reshape(-1,)
test_label = data_label_all[test_index]

train = data_all[train_index,:,:,:]
val = data_all[val_index,:,:,:]
test = data_all[test_index,:,:,:]

split_data.data_info(train_label, val_label, test_label, np.max(data_label_all))

class 0 	 0 	 0 	 0
class 1 	 10 	 5 	 31
class 2 	 10 	 5 	 1413
class 3 	 10 	 5 	 815
class 4 	 10 	 5 	 222
class 5 	 10 	 5 	 468
class 6 	 10 	 5 	 715
class 7 	 10 	 5 	 13
class 8 	 10 	 5 	 463
class 9 	 10 	 5 	 5
class 10 	 10 	 5 	 957
class 11 	 10 	 5 	 2440
class 12 	 10 	 5 	 578
class 13 	 10 	 5 	 190
class 14 	 10 	 5 	 1250
class 15 	 10 	 5 	 371
class 16 	 10 	 5 	 78
total     	 160 	 80 	 10009


In [10]:
train = train.reshape(-1, img_size, img_size, bands)
val = val.reshape(-1, img_size, img_size, bands)
test = test.reshape(-1,img_size, img_size, bands)
print('before transpose: train shape: ', train.shape)
print('before transpose: test  shape: ', val.shape)
print('before transpose: test  shape: ', test.shape)

before transpose: train shape:  (160, 33, 33, 30)
before transpose: test  shape:  (80, 33, 33, 30)
before transpose: test  shape:  (10009, 33, 33, 30)


In [11]:
train = train.transpose(0, 3, 1, 2)
val = val.transpose(0,  3, 1,2 )
test = test.transpose(0,  3, 1,2 )
print('after transpose: train shape: ', train.shape)
print('after transpose: val  shape: ', val.shape)
print('after transpose: test  shape: ', test.shape)

after transpose: train shape:  (160, 30, 33, 33)
after transpose: val  shape:  (80, 30, 33, 33)
after transpose: test  shape:  (10009, 30, 33, 33)


In [12]:
train_hi_gt= torch.zeros(train_label.size,class_num)
val_hi_gt= torch.zeros(val_label.size,class_num)
test_hi_gt= torch.zeros(test_label.size,class_num)


train_hi_gt = torch.LongTensor(train_hi_gt.numpy())
test_hi_gt = torch.LongTensor(test_hi_gt.numpy())
val_hi_gt = torch.LongTensor(val_hi_gt.numpy())

class TrainDS(torch.utils.data.Dataset):
    def __init__(self):
        self.len = train.shape[0]
        self.x_data = torch.FloatTensor(train)
        self.y_data = torch.LongTensor(train_label)
    def __getitem__(self, index):
        # 根据索引返回数据和对应的标签
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        # 返回文件数据的数目
        return self.len


""" Val dataset"""


class ValDS(torch.utils.data.Dataset):
    def __init__(self):
        self.len = val.shape[0]
        self.x_data = torch.FloatTensor(val)
        self.y_data = torch.LongTensor(val_label)

    def __getitem__(self, index):
        # 根据索引返回数据和对应的标签
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        # 返回文件数据的数目
        return self.len


""" Testing dataset"""


class TestDS(torch.utils.data.Dataset):
    def __init__(self):
        self.len = test.shape[0]
        self.x_data = torch.FloatTensor(test)
        self.y_data = torch.LongTensor(test_label)  
    def __getitem__(self, index):
        # 根据索引返回数据和对应的标签
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        # 返回文件数据的数目
        return self.len


# 创建 trainloader 和 testloader
trainset = TrainDS()
valset = ValDS()
testset = TestDS()
train_loader = torch.utils.data.DataLoader(dataset=trainset, batch_size=32, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(dataset=valset, batch_size=32, shuffle=False, num_workers=0)
test_loader = torch.utils.data.DataLoader(dataset=testset, batch_size=32, shuffle=False, num_workers=0)

In [13]:
model = ss_fusion_cls.SSFusionFramework(
                img_size = img_size,
                in_channels = pca_components,
                patch_size=patch_size,
                classes = class_num+1,
                model_size='base' #The optional values are 'base','large' and 'huge'
)


In [14]:
model_params =model.state_dict()
spat_net = torch.load((r"/home5/wsq/HyperSIGMA/spat-base.pth"), map_location=torch.device('cuda'))
for k in list(spat_net['model'].keys()):
    if 'patch_embed.proj' in k:
        del spat_net['model'][k]
for k in list(spat_net['model'].keys()):
    if 'spat_map' in k:
        del spat_net['model'][k]
for k in list(spat_net['model'].keys()):
    if 'spat_output_maps' in k:
        del spat_net['model'][k]
for k in list(spat_net['model'].keys()):
    if 'pos_embed' in k:
        del spat_net['model'][k]
spat_weights = {}
prefix = 'spat_encoder.'
for key, value in spat_net['model'].items():
    new_key = prefix + key
    spat_weights[new_key] = value
per_net = torch.load((r"/home5/wsq/HyperSIGMA/spec-base.pth"), map_location=torch.device('cuda'))
model_params =model.state_dict()
for k in list(per_net['model'].keys()):
    if 'patch_embed.proj' in k:
        del per_net['model'][k]
    if 'spat_map' in k:
        del per_net['model'][k]
    if 'fpn1.0.weight' in k:
        del per_net['model'][k]
spec_weights = {}
prefix = 'spec_encoder.'
for key, value in per_net['model'].items():
    new_key = prefix + key
    spec_weights[new_key] = value
model_params =model.state_dict()
for k in list(spec_weights.keys()):
    if 'spec_encoder.patch_embed' in k:
        del spec_weights[k]
merged_params = {**spat_weights, **spec_weights}
same_parsms = {k: v for k, v in merged_params.items() if k in model_params.keys()}
model_params.update(same_parsms)
model.load_state_dict(model_params)

/tmp/ipykernel_1678656/1626247618.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  spat_net = torch.load((r"/home5/wsq/HyperSIGMA/spat-base.pth"), map_location=torch.devi

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 44.56 GiB of which 11.31 MiB is free. Process 3894991 has 257.00 MiB memory in use. Process 2675314 has 257.00 MiB memory in use. Process 1589778 has 40.91 GiB memory in use. Including non-PyTorch memory, this process has 3.14 GiB memory in use. Of the allocated memory 2.13 GiB is allocated by PyTorch, and 212.33 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
optim_wrapper = dict(
    optimizer=dict(
    type='AdamW', lr=6e-5, betas=(0.9, 0.999), weight_decay=0.05),
    constructor='LayerDecayOptimizerConstructor_ViT', 
    paramwise_cfg=dict(
        num_layers=12, 
        layer_decay_rate=0.9,
        )
        )
optimizer = build_optim_wrapper(model, optim_wrapper)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer.optimizer, 300, eta_min=0, last_epoch=-1)
criterion = nn.CrossEntropyLoss()
model.to(device)
count = 0
best_loss = 99999
train_losses = []
val_losses = []

In [ ]:
for epoch in range(max_epoch+ 1):
    correct = 0
    total = 0
    test_correct = 0
    test_total = 0
    _train_loss = 0
    val_correct = 0
    model.train()
    for x, y in train_loader:
        if torch.cuda.is_available():
            x, y = x.to(device), y.to(device)
        y_pred = model(x)
        train_loss = criterion(y_pred, y)
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        _train_loss += train_loss.item()

        with torch.no_grad():
            y_pred = torch.argmax(y_pred, dim=1)
            correct = (y_pred == y).sum().item()
            total = y.size(0)

    train_losses.append(train_loss.cpu().detach().item())
    
    if epoch % 10==0:
       
        epoch_loss = _train_loss / len(train_loader.dataset)
        epoch_accuracy = correct / total
        if epoch_loss < best_loss:
            best_loss = epoch_loss

            # 检查并创建目录
            #os.makedirs(path_weight, exist_ok=True)

            torch.save(model.state_dict(), path_weight + "model.pth")
            torch.save(optimizer.state_dict(), path_weight + 'optimizer.pth')
            print('save model')

        
        print('epoch', epoch,
            'loss: ', round(epoch_loss, 5),
            'accuracy: ', round(epoch_accuracy, 5),
            )

In [ ]:
count = 0
model.load_state_dict(torch.load(path_weight + r"model.pth"))
model.eval()
with torch.no_grad():
    for x, y  in test_loader:
        if torch.cuda.is_available():
            x, y = x.to(device), y.to(device)
        output = model(x)
        y_pred = torch.argmax(output, dim=1)
        test_correct += (y_pred == y).sum().item()
        test_total += y.size(0)

        if count == 0:
            y_pred_test =  y_pred.cpu().numpy()
            y_gt_test = y.cpu().numpy()
            count = 1
        else:
            y_pred_test = np.concatenate( (y_pred_test, y_pred.cpu().numpy()) )
            y_gt_test = np.concatenate( (y_gt_test, y.cpu().numpy()) )

overall_acc = metrics.accuracy_score(y_pred_test, y_gt_test)
confusion_matrix = metrics.confusion_matrix(y_pred_test, y_gt_test)
each_acc, average_acc = utils.aa_and_each_accuracy(confusion_matrix)
kappa = metrics.cohen_kappa_score(y_pred_test, y_gt_test)
OA_HybirdSN = test_correct / test_total
print("test OA={:.4f}".format(overall_acc))
print('kappa=',kappa)
print('each_acc=',each_acc)
print('average_acc=',average_acc)